In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import validators

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import urllib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from tqdm import tqdm

In [2]:
nfl_data = pd.read_csv("./nfl-elo/nfl_elo.csv")
nfl_data['year'] = nfl_data['date'].apply(lambda x: int(x.split("-")[0]))
#nfl_data['month'] = nfl_data['date'].apply(lambda x: int(x.split("-")[1]))

game_data = nfl_data[(nfl_data['year'] == 2010) | \
                     (nfl_data['year'] == 2011) | \
                     (nfl_data['year'] == 2012) | \
                     (nfl_data['year'] == 2013) | \
                     (nfl_data['year'] == 2014) | \
                     (nfl_data['year'] == 2015) | \
                     (nfl_data['year'] == 2016) | \
                     (nfl_data['year'] == 2017) | \
                     (nfl_data['year'] == 2018) | \
                     (nfl_data['year'] == 2019) | \
                     (nfl_data['year'] == 2020) | \
                     (nfl_data['year'] == 2021) | \
                     (nfl_data['year'] == 2022)
                    ]
game_data = game_data[27:]

In [67]:
teams = [
    'BUF',
    'BAL',
    'HOU',
    'DEN',
    'MIA',
    'CIN',
    'IND',
    'KAN',
    'NWE',
    'CLE',
    'JAX',
    'LVR',
    'NYJ',
    'PIT',
    'TEN',
    'LAC',
    'DAL',
    'CHI',
    'ATL',
    'ARI',
    'NYG',
    'DET',
    'CAR',
    'LAR',
    'PHI',
    'GBP',
    'NOS',
    'SFF',
    'WAS',
    'MIN',
    'TBB',
    'SEA'
]

teams_map = {
    'Buffalo Bills': 'BUF',
    'Baltimore Ravens': 'BAL',
    'Houston Texans': 'HOU',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Cincinnati Bengals': 'CIN',
    'Indianapolis Colts': 'IND',
    'Kansas City Chiefs': 'KAN',
    'New England Patriots': 'NWE',
    'Cleveland Browns': 'CLE',
    'Jacksonville Jaguars': 'JAX',
    'Las Vegas Raiders': 'LVR',
    'Oakland Raiders': 'LVR',
    'New York Jets': 'NYJ',
    'Pittsburgh Steelers': 'PIT',
    'Tennessee Titans': 'TEN',
    'Los Angeles Chargers': 'LAC',
    'San Diego Chargers': 'LAC',
    'Dallas Cowboys': 'DAL',
    'Chicago Bears': 'CHI',
    'Atlanta Falcons': 'ATL',
    'Arizona Cardinals': 'ARI',
    'New York Giants': 'NYG',
    'Detroit Lions': 'DET',
    'Carolina Panthers': 'CAR',
    'Los Angeles Rams': 'LAR',
    'St Louis Rams': 'LAR',
    'Philadelphia Eagles': 'PHI',
    'Green Bay Packers': 'GBP',
    'New Orleans Saints': 'NOS',
    'San Francisco 49ers': 'SFF',
    'Washington Football Team': 'WAS',
    'Washington Redskins': 'WAS',
    'Minnesota Vikings': 'MIN',
    'Tampa Bay Buccaneers': 'TBB',
    'Seattle Seahawks': 'SEA'
}


url_elo_team_map = {
    'BUF': 'BUF',
    'BAL': 'BAL',
    'HOU': 'HOU',
    'DEN': 'DEN',
    'MIA': 'MIA',
    'CIN': 'CIN',
    'IND': 'IND',
    'KAN': 'KC',
    'NWE': 'NWE',
    'CLE': 'CLE',
    'JAX': 'JAX',
    'LVR': 'OAK',
    'NYJ': 'NYJ',
    'PIT': 'PIT',
    'TEN': 'TEN',
    'LAC': 'LAC',
    'DAL': 'DAL',
    'CHI': 'CHI',
    'ATL': 'ATL',
    'ARI': 'ARI',
    'NYG': 'NYG',
    'DET': 'DET',
    'CAR': 'CAR',
    'LAR': 'LAR',
    'PHI': 'PHI',
    'GBP': 'GB',
    'NOS': 'NO',
    'SFF': 'SF',
    'WAS': 'WSH',
    'MIN': 'MIN',
    'TBB': 'TB',
    'SEA': 'SEA'
}

month_map = {"Jan": 1, "Feb": 2, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [4]:
base_url = "https://www.sportsoddshistory.com/nfl-game-team/?tm="
previous_years = "&d=2010#2019"
valid = True
for t in teams:
    validators.url(base_url + t)
    if(not validators.url(base_url + t)):
        print("{} IS NOT VALID".format(t))
        valid = False
    if(not validators.url(base_url + t + previous_years)):
        print("{} IS NOT VALID".format(t))
        valid = False
        
if(valid):
    print("ALL URLS ARE VALID")

ALL URLS ARE VALID


## Put scraper together for all teams

In [53]:
def scrape_tables(team, tables):
    dates = []
    times = []
    current_teams = []
    opponents = []
    team_scores = []
    opp_scores = []
    game_results = []
    spreads = []
    spread_results = []
    ou_lines = []
    ou_results = []
    for table in tables:
        th_val = len(table.findAll('th'))
        if(th_val in [9,10]):
            #for i in range(len(table.findAll('td'))//th_val):
            for i in range(0, len(table.findAll('td')), th_val):

                table_td = table.findAll('td')

                # Get Date and time
                #date_td = table_td[i*th_val + 2].contents[0]
                #print(table_td[i+2].contents[0])
                date_td = table_td[i + 2].contents[0]
                date_str = date_td[-4:] + "-"
                #month_str = str(month_map[table.findAll('td')[i*th_val + 2].contents[0][:3]])
                month_str = str(month_map[date_td[:3]])
                month_str = month_str if(len(month_str) == 2) else '0'+month_str
                date_str += month_str
                day_str = date_td.split(" ")[1].split(",")[0]
                day_str = day_str if(len(day_str) == 2) else '0'+day_str
                date_str += "-" + day_str
                dates.append(date_str)
                #times.append(table_td[i*th_val + 3].contents[0])
                times.append(table_td[i + 3].contents[0])


                # Get teams
                current_teams.append(team)
                #opponents.append(teams_map[table_td[i*th_val + 5].contents[0].contents[0]])
                opponents.append(teams_map[table_td[i + 5].contents[0].contents[0]])

                # Get score and outcome
                #game_results.append(table_td[i*th_val + 6].contents[0][0])
                #team_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[0]))
                #opp_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[1].split(" ")[0]))
                game_results.append(table_td[i + 6].contents[0][0])
                team_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[0]))
                opp_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[1].split(" ")[0]))

                # Get spread and outcome
                #spread_results.append(table_td[i*th_val + 7].contents[0][0])
                spread_results.append(table_td[i + 7].contents[0][0])
                #if(table_td[i*th_val + 7].contents[0][2:] == 'PK'):
                if(table_td[i + 7].contents[0][2:] == 'PK'):
                    spreads.append(0.)
                else:
                    #spreads.append(float(table_td[i*th_val + 7].contents[0][2:]))
                    spreads.append(float(table_td[i + 7].contents[0][2:]))

                # Get over/under and outcome
                #ou_results.append(table_td[i*th_val + 8].contents[0][0])
                #ou_lines.append(table_td[i*th_val + 8].contents[0][2:])
                ou_results.append(table_td[i + 8].contents[0][0])
                ou_lines.append(table_td[i + 8].contents[0][2:])

    return dates, times, current_teams, opponents, game_results, team_scores, opp_scores, spread_results, spreads, ou_results, ou_lines

In [54]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []

for idx, t in tqdm(enumerate(teams)):
    url = base_url + t #+ previous_years
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)

    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2020-2021 {}".format(t))
    
    time.sleep(1)
    url = base_url + t + previous_years
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)
    
    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2010-2019 {}".format(t))

    time.sleep(5)
    
print(dates)

0it [00:00, ?it/s]

https://www.sportsoddshistory.com/nfl-game-team/?tm=BUF
DONE WITH 2020-2021 BUF
DONE WITH 2010-2019 BUF


1it [00:10, 10.42s/it]

https://www.sportsoddshistory.com/nfl-game-team/?tm=BAL
DONE WITH 2020-2021 BAL
DONE WITH 2010-2019 BAL


2it [00:20,  9.95s/it]

...

https://www.sportsoddshistory.com/nfl-game-team/?tm=TBB
DONE WITH 2020-2021 TBB
DONE WITH 2010-2019 TBB


31it [06:06, 11.58s/it]

https://www.sportsoddshistory.com/nfl-game-team/?tm=SEA
DONE WITH 2020-2021 SEA
DONE WITH 2010-2019 SEA


32it [06:17, 11.78s/it]

['2021-09-12', '2021-09-19', '2021-09-26', '2021-10-03', '2021-10-10', '2021-10-18', '2021-10-31', '2021-11-07', '2021-11-14', '2021-11-21', '2021-11-25', '2021-12-06', '2021-12-12', '2021-12-19', '2021-12-26', '2022-01-02', '2022-01-09', '2022-01-15', '2022-01-23', '2020-09-13', '2020-09-20', '2020-09-27', '2020-10-04', '2020-10-13', '2020-10-19', '2020-10-25', '2020-11-01', '2020-11-08', '2020-11-15', '2020-11-29', '2020-12-07', '2020-12-13', '2020-12-19', '2020-12-28', '2021-01-03', '2021-01-09', '2021-01-16', '2021-01-24', '2019-09-08', '2019-09-15', '2019-09-22', '2019-09-29', '2019-10-06', '2019-10-20', '2019-10-27', '2019-11-03', '2019-11-10', '2019-11-17', '2019-11-24', '2019-11-28', '2019-12-08', '2019-12-15', '2019-12-21', '2019-12-29', '2020-01-04', '2018-09-09', '2018-09-16', '2018-09-23', '2018-09-30', '2018-10-07', '2018-10-14', '2018-10-21', '2018-10-29', '2018-11-04', '2018-11-11', '2018-11-25', '2018-12-02', '2018-12-09', '2018-12-16', '2018-12-23', '2018-12-30', '2017

In [56]:
# Save data
np.save("./scraped_data/dates.npy", dates)
np.save("./scraped_data/times.npy", times)
np.save("./scraped_data/current_teams.npy", current_teams)
np.save("./scraped_data/opponents.npy", opponents)
np.save("./scraped_data/game_results.npy", game_results)
np.save("./scraped_data/team_scores.npy", team_scores)
np.save("./scraped_data/opp_scores.npy", opp_scores)
np.save("./scraped_data/spread_results.npy", spread_results)
np.save("./scraped_data/spreads.npy", spreads)
np.save("./scraped_data/ou_results.npy", ou_results)
np.save("./scraped_data/ou_lines.npy", ou_lines)

## Make dataset

In [57]:
scraped_data = pd.DataFrame()
scraped_data['date'] = dates
scraped_data['time'] = times
scraped_data['current_team'] = current_teams
scraped_data['opponent'] = opponents
scraped_data['game_result'] = game_results
scraped_data['team_score'] = team_scores
scraped_data['opp_score'] = opp_scores
scraped_data['spread_result'] = spread_results
scraped_data['spread'] = spreads
scraped_data['ou_result'] = ou_results
scraped_data['ou_line'] = ou_lines
scraped_data['year'] = scraped_data['date'].apply(lambda x: int(x.split("-")[0]))
min_2010 = scraped_data[(scraped_data['current_team'] == 'MIN')]
elo_min_2010 = game_data[(game_data['year'] == 2010) & ((game_data['team1']=='MIN') | (game_data['team2']=='MIN'))]

#print(min_2010['date'])
#print(elo_min_2010['date'])
#print(set(scraped_data['date'].values) & set(game_data['date'].values))

#for d in sorted(np.unique(scraped_data['date'].values)):
#    print(d)
#print("\n\n")
#for d in sorted(np.unique(game_data['date'].values)):
#    print(d)
    
print(len(sorted(np.unique(scraped_data['date'].values))))
print(len(sorted(np.unique(game_data['date'].values))))
#for g in game_data[['date', 'team1', 'team2']].iterrows():
#    try:
#        print(dates == g[1]['date'])
#    except:
#        continue

660
663


In [152]:
no_match, match = 0, 0
#print(scraped_data['date'].values)
scraped_data['date'] = scraped_data['date'].apply(lambda x: x[:-1]+"0"+x[-1:] if(len(x) == 9) else x)

#print(game_data[((game_data['team1'] == 'BAL') | (game_data['team2']=='BAL')) & (game_data['year']==2021)])

#print(set(game_data['team1'].values))

betting_data = game_data.copy(deep=True)
betting_data['spread'] = np.nan         # Spread for the home team
betting_data['spread_result'] = np.nan  # Indicates whether or not the home team beat the spread
betting_data['ou_line'] = np.nan        # OU line for combined score
betting_data['ou_result'] = np.nan      # Indicates whether or not OU was hit

for idx, row in tqdm(scraped_data.iterrows()):
    date_data = betting_data[betting_data['date'] == row['date']]
    #print(row['date'])
    current_team = url_elo_team_map[row['current_team']]
    opponent = url_elo_team_map[row['opponent']]
    home_match = date_data[(date_data['team1'] == current_team)] 
    away_match = date_data[(date_data['team2'] == current_team)] 
    home_opp = date_data[(date_data['team1'] == opponent)]
    away_opp = date_data[(date_data['team2'] == opponent)]
    if(not home_match.empty):
        betting_data['ou_line'][home_match.index[0]] = row['ou_line']
        betting_data['ou_result'][home_match.index[0]] = row['ou_result']
        betting_data['spread'][home_match.index[0]] = row['spread']
        betting_data['spread_result'][home_match.index[0]] = row['spread_result']
    elif(not away_match.empty):
        betting_data['ou_line'][away_match.index[0]] = row['ou_line']
        betting_data['ou_result'][away_match.index[0]] = row['ou_result']
        spread = np.abs(row['spread']) if(row['spread'] < 0) else -row['spread']
        spread_result = 'L' if(row['spread_result']=='W') else 'W' if(row['spread_result']=='L') else row['spread_result']
        betting_data['spread'][away_match.index[0]] = spread
        betting_data['spread_result'][away_match.index[0]] = spread_result
        print(row['spread'], spread)
        print(row['spread_result'], spread_result)
    #elif(not home_opp.empty):
    #    betting_data['ou_line'][home_opp.index[0]] = row['ou_line']
    #    betting_data['ou_result'][home_opp.index[0]] = row['ou_result']
    #    print(row['spread'])
    #    print(row['spread_result'])
    #    raise
    #elif(not away_opp.empty):
    #    betting_data['ou_line'][away_opp.index[0]] = row['ou_line']
    #    betting_data['ou_result'][away_opp.index[0]] = row['ou_result']
    #    betting_data['spread'][away_opp.index[0]] = row['spread']
    #    betting_data['spread_result'][away_opp.index[0]] = row['spread_result']
    else:
        continue

0it [00:00, ?it/s]/var/folders/mh/wfzfv8nd3g7_8w30_pjfbhtr0000gn/T/ipykernel_10888/3890882755.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betting_data['ou_line'][home_match.index[0]] = row['ou_line']
/var/folders/mh/wfzfv8nd3g7_8w30_pjfbhtr0000gn/T/ipykernel_10888/3890882755.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betting_data['ou_result'][home_match.index[0]] = row['ou_result']
/var/folders/mh/wfzfv8nd3g7_8w30_pjfbhtr0000gn/T/ipykernel_10888/3890882755.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

-3.5 3.5
W L
2.5 -2.5
W L
-6.0 6.0
L W
-14.5 14.5
L W
-13.0 13.0
W L
-6.5 6.5
W L
3.5 -3.5
L W
1.5 -1.5
W L
2.0 -2.0
L W
-5.5 5.5
L W


37it [00:00, 93.09it/s]

-3.0 3.0
W L
-3.0 3.0
L W
-9.5 9.5
L W
3.0 -3.0
W L
1.5 -1.5
W L
-5.5 5.5
W L
-7.0 7.0
W L
3.0 -3.0
L W
2.5 -2.5
W L
-1.0 1.0
W L


57it [00:00, 95.28it/s]

...

2.5 -2.5
W L
-3.0 3.0
L W
-2.5 2.5
L W
2.0 -2.0
W L
7.5 -7.5
W L
3.0 -3.0
L W
-3.0 3.0
L W
3.5 -3.5
W L
-5.0 5.0
W L
-3.0 3.0
W L
3.0 -3.0
W L
6.0 -6.0
L W


5608it [01:00, 95.20it/s]

14.0 -14.0
L W
10.0 -10.0
W L
3.0 -3.0
P P
11.0 -11.0
W L
3.0 -3.0
W L
3.5 -3.5
W L
2.5 -2.5
L W
3.5 -3.5
L W
-2.0 2.0
L W
6.0 -6.0
W L


5628it [01:00, 93.24it/s]

2.0 -2.0
L W
4.0 -4.0
W L
11.0 -11.0
L W
5.0 -5.0
L W
6.0 -6.0
L W
10.0 -10.0
L W


In [153]:
print(len(betting_data['ou_line'].dropna()))
print(len(betting_data['ou_result'].dropna()))
print(len(betting_data['spread'].dropna()))
print(len(betting_data['spread_result'].dropna()))
print(len(betting_data['ou_line']))

#print(betting_data.loc[13872])
#missing = betting_data.loc[13872][['date', 'team1']].values

#print(missing)
#print(scraped_data[scraped_data['date'] == missing[0]])

3197
3197
3197
3197
3224


## Debugging code

In [47]:
url = base_url + teams[1] #+ previous_years
print(url)
req = requests.get(url)#, headers)
soup = BeautifulSoup(req.content, 'html.parser')
tables = soup.findAll(lambda tag: tag.name=='table')

https://www.sportsoddshistory.com/nfl-game-team/?tm=BAL


In [48]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []
for table in tables:
    th_val = len(table.findAll('th'))
    if(th_val in [9,10]):
        print(len(table.findAll('td')))
        #for i in range(len(table.findAll('td'))//th_val):
        for i in range(0, len(table.findAll('td')), th_val):
            print(i)
            table_td = table.findAll('td')
            #print(table_td[i*th_val:(i+1)*th_val])
            
            # Get Date and time
            date_td = table_td[i + 2].contents[0]
            date_str = date_td[-4:] + "-"
            #if(date_str not in ['2010-', '2011-']):
            #    continue
            month_str = str(month_map[table.findAll('td')[i + 2].contents[0][:3]])
            month_str = month_str if(len(month_str) == 2) else '0'+month_str
            date_str += month_str
            date_str += "-" + date_td.split(" ")[1].split(",")[0]
            print(date_str)
            dates.append(date_str)
            times.append(table_td[i + 3].contents[0])

            
            # Get teams
            current_teams.append("BUF")
            opponents.append(teams_map[table_td[i + 5].contents[0].contents[0]])
            
            # Get score and outcome
            game_results.append(table_td[i + 6].contents[0][0])
            team_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[0]))
            opp_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[1].split(" ")[0]))

            # Get spread and outcome
            spread_results.append(table_td[i + 7].contents[0][0])
            if(table_td[i + 7].contents[0][2:] == 'PK'):
                spreads.append(0.)
            else:
                spreads.append(float(table_td[i + 7].contents[0][2:]))
            
            # Get over/under and outcome
            ou_results.append(table_td[i + 8].contents[0][0])
            ou_lines.append(table_td[i + 8].contents[0][2:])
            
print(dates)
print(times)
print(opponents)
print(game_results)
print(team_scores)
print(opp_scores)
print(spread_results)
print(spreads)
print(ou_results)
print(len(ou_lines))

170
0
2021-09-13
10
2021-09-19
20
2021-09-26
30
2021-10-3
40
2021-10-11
50
2021-10-17
60
2021-10-24
70
2021-11-7
80
2021-11-11
90
2021-11-21
100
2021-11-28
110
2021-12-5
120
2021-12-12
130
2021-12-19
140
2021-12-26
150
2022-01-2
160
2022-01-9
160
0
2020-09-13
10
2020-09-20
20
2020-09-28
30
2020-10-4
40
2020-10-11
50
2020-10-18
60
2020-11-1
70
2020-11-8
80
2020-11-15
90
2020-11-22
100
2020-12-2
110
2020-12-8
120
2020-12-14
130
2020-12-20
140
2020-12-27
150
2021-01-3
18
0
2021-01-10
9
2021-01-16
['2021-09-13', '2021-09-19', '2021-09-26', '2021-10-3', '2021-10-11', '2021-10-17', '2021-10-24', '2021-11-7', '2021-11-11', '2021-11-21', '2021-11-28', '2021-12-5', '2021-12-12', '2021-12-19', '2021-12-26', '2022-01-2', '2022-01-9', '2020-09-13', '2020-09-20', '2020-09-28', '2020-10-4', '2020-10-11', '2020-10-18', '2020-11-1', '2020-11-8', '2020-11-15', '2020-11-22', '2020-12-2', '2020-12-8', '2020-12-14', '2020-12-20', '2020-12-27', '2021-01-3', '2021-01-10', '2021-01-16']
['8:15', '8:20', '1:0